#Imports:

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD,Adam
from keras.models import load_model
from keras.layers import Embedding, LSTM,GRU

import random
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Loading Data:

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
words=[]
tags = []
documents = []
puncc = ['?', '!']
df = open('intents.json').read()
intents = json.loads(df)

In [ ]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Is anyone there?',
    'Hello',
    'Good day',
    "What's up",
    'how are ya',
    'heyy',
    'whatsup',
    '??? ??? ??'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'see you',
    'bye bye',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Bye',
    'Have a Good day',
    'talk to you later',
    'ttyl',
    'i got to go',
    'gtg'],
   'responses': ['Sad to see you go :(',
    'Talk to you later',
    'Goodbye!',
    'Come back soon'],
   'context_set': ''},
  {'tag': 'creator',
   'patterns': ['what is the name of your developers',
    'what is the name of your creators',
    'what is the name of the developers',
    'what is the name of the creators',
    'who created you',
    'your developers',
    'your creators',
    'who are your developers',
    'developers',
  

#Pre-Processing:

In [ ]:
for i in intents['intents']:
    for p in i['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(p)
        words.extend(w)
        documents.append((w, i['tag']))
        if i['tag'] not in tags:
            tags.append(i['tag'])

In [ ]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in puncc]
words = sorted(list(set(words)))
# sort classes
tags = sorted(list(set(tags)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(tags), "classes", tags)
# words = all words
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(tags,open('classes.pkl','wb'))

405 documents
38 classes ['admission', 'canteen', 'college intake', 'committee', 'computerhod', 'course', 'creator', 'document', 'event', 'extchod', 'facilities', 'fees', 'floors', 'goodbye', 'greeting', 'hod', 'hostel', 'hours', 'infrastructure', 'ithod', 'library', 'location', 'menu', 'name', 'number', 'placement', 'principal', 'ragging', 'random', 'salutaion', 'scholarship', 'sem', 'sports', 'swear', 'syllabus', 'task', 'uniform', 'vacation']
263 unique lemmatized words ["'s", '(', ')', 'a', 'about', 'ac', 'active', 'activity', 'address', 'admision', 'admission', 'against', 'ai/ml', 'allotment', 'am', 'an', 'and', 'antiragging', 'any', 'anyone', 'are', 'as', 'asshole', 'at', 'attend', 'automobile', 'available', 'average', 'be', 'between', 'big', 'bitch', 'book', 'boy', 'branch', 'bring', 'building', 'by', 'bye', 'cafetaria', 'call', 'called', 'campus', 'can', 'canteen', 'capacity', 'case', 'casuals', 'ce', 'chatting', 'chemical', 'civil', 'code', 'college', 'come', 'committe', 'comm

In [ ]:
train_df = []
output = [0] * len(tags)
for d in documents:
    # initialize  bag of words
    bag = []
    pattern_words = d[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output)
    output_row[tags.index(d[1])] = 1
    
    train_df.append([bag, output_row])


In [ ]:
random.shuffle(train_df)
train_df = np.array(train_df)
train_x = list(train_df[:,0])
train_y = list(train_df[:,1])

<ipython-input-7-5d3a8ab30cb5>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_df = np.array(train_df)


## Modeling:

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
81/81 [==============================] - 2s 5ms/step - loss: 3.6198 - accuracy: 0.0420
Epoch 2/200
81/81 [==============================] - 0s 5ms/step - loss: 3.4784 - accuracy: 0.1111
Epoch 3/200
81/81 [==============================] - 0s 5ms/step - loss: 3.2498 - accuracy: 0.1679
Epoch 4/200
81/81 [==============================] - 0s 5ms/step - loss: 3.0354 - accuracy: 0.2099
Epoch 5/200
81/81 [==============================] - 1s 7ms/step - loss: 2.7689 - accuracy: 0.2938
Epoch 6/200
81/81 [==============================] - 0s 5ms/step - loss: 2.5312 - accuracy: 0.3556
Epoch 7/200
81/81 [==============================] - 1s 6ms/step - loss: 2.2932 - accuracy: 0.4222
Epoch 8/200
81/81 [==============================] - 0s 6ms/step - loss: 2.0674 - accuracy: 0.4617
Epoch 9/200
81/81 [==============================] - 0s 5ms/step - loss: 1.8666 - accuracy: 0.5259
Epoch 10/200
81/81 [==============================] - 1s 6ms/step - loss: 1.7030 - accuracy: 0.5605
Epoch 11/

In [ ]:
model.save('chatbot_model.h5', hist)

In [ ]:
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

# Helper Functions:

In [ ]:
def clean_input(input):
    sentence_words = nltk.word_tokenize(input)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(input, words, show_details=True):

    sentence_words = clean_input(input)
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_tag(input, model):
    p = bow(input, words,show_details=False)
    res = model.predict(np.array([p]),verbose=0)[0];
    results = [[i,r] for i,r in enumerate(res) ]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_answer(text):
    ints = predict_tag(text, model)
    res = getResponse(ints, intents)
    return res


In [ ]:
msg="hi I am aya"
res = chatbot_answer(msg)
print(res)

Good to see you again!


In [ ]:
msg="how are you?"
res = chatbot_answer(msg)
print(res)

Hi there, how can I help?


In [ ]:
msg="what is your name"
res = chatbot_answer(msg)
print(res)

I am a Chatbot.


In [ ]:
msg="?????"
res = chatbot_answer(msg)
print(res)

Hi there, how can I help?


In [ ]:
msg="Does college have canteen"
res = chatbot_answer(msg)
print(res)

Our university has canteen with variety of food available


In [ ]:
msg="holiday list"
res = chatbot_answer(msg)
print(res)


Academic calender is given to you by your class-soordinators after you join your respective classes


In [ ]:
msg="sports facilities"
res = chatbot_answer(msg)
print(res)

Our university encourages all-round development of students and hence provides sports facilities in the campus. For more details visit<a target="_blank" href=/"(LINK IF HAVE)">here</a>


In [ ]:
msg="bye"
res = chatbot_answer(msg)
print(res)

Goodbye!
